In [ ]:
class SoilTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Create DataLoader for test data
test_dataset = SoilTestDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load label mappings (must be consistent with training)
label_to_index_mapping = {'Alluvial soil': 0, 'Black soil': 1, 'Clay soil': 2, 'Red soil': 3}
index_to_label_mapping = {v: k for k, v in label_to_index_mapping.items()}

# Load the model architecture and weights
model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, len(label_to_index_mapping))
model.load_state_dict(torch.load("best_soil_model.pth", map_location=device))
model.to(device)
model.eval()

predicted_indices = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predicted_indices.extend(preds.cpu().numpy())

# Map predictions back to soil type labels
test_df['soil_type'] = [index_to_label_mapping[idx] for idx in predicted_indices]

# Save submission file
submission = test_df[['image_id', 'soil_type']]
submission.to_csv("submission.csv", index=False)

print("✅ Submission file saved as 'submission.csv'")
